In [1]:
import os
import sys
import torch
import argparse
import torchvision
import numpy as np
import torch.nn as nn
import scipy.io as sio
import cv2
import pywt
import pywt.data
import matplotlib.image as image

from PIL import Image
from tqdm import tqdm
from datetime import datetime
from collections import defaultdict
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from os.path import join, isdir, abspath, dirname
import matplotlib.pyplot as plt
# Customized import.
from networks import HED
from datasets import BsdsDataset
from utils import Logger, AverageMeter, \
    load_checkpoint, save_checkpoint, load_vgg16_caffe, load_pretrained_caffe

In [2]:
list_path = './data/HED-BSDS/train_pair.lst'

In [3]:
with open(list_path, 'r') as f:
    lines = f.readlines()
lines = [line.strip() for line in lines]  # Remove the newline at last.
pairs = [line.split() for line in lines]
images_path = [pair[0] for pair in pairs]
edges_path  = [pair[1] for pair in pairs]

In [4]:
images_path_LL = []
images_path_LH = []
images_path_HL = []
images_path_HH = []


for i in images_path:
    
    i_LL = 'LL/' + i
    i_LH = 'LH/' + i
    i_HL = 'HL/' + i
    i_HH = 'HH/' + i

    images_path_LL.append(i_LL)
    images_path_LH.append(i_LH)
    images_path_HL.append(i_HL)
    images_path_HH.append(i_HH)
    
edges_path_LL = []
edges_path_LH = []
edges_path_HL = []
edges_path_HH = []


for i in edges_path:
    
    i_LL = 'LL/' + i
    i_LH = 'LH/' + i
    i_HL = 'HL/' + i
    i_HH = 'HH/' + i

    edges_path_LL.append(i_LL)
    edges_path_LH.append(i_LH)
    edges_path_HL.append(i_HL)
    edges_path_HH.append(i_HH)

In [7]:
LL_list_path = list_path[:list_path.rfind('/')+1] + 'LL/' + list_path[list_path.rfind('/')+1:]
LH_list_path = list_path[:list_path.rfind('/')+1] + 'LH/' + list_path[list_path.rfind('/')+1:]
HL_list_path = list_path[:list_path.rfind('/')+1] + 'HL/' + list_path[list_path.rfind('/')+1:]
HH_list_path = list_path[:list_path.rfind('/')+1] + 'HH/' + list_path[list_path.rfind('/')+1:]


with open(LL_list_path, 'w') as f:
    for i in range(len(images_path_LL)):
#         print(images_path_LL[i] + ' ' + edges_path_LL[i] + '\n')
        f.write(images_path_LL[i] + ' ' + edges_path_LL[i] + '\n')

In [9]:

with open(LH_list_path, 'w') as f:
    for i in range(len(images_path_LH)):
        f.write(images_path_LH[i] + ' ' + edges_path_LH[i] + '\n')

with open(HL_list_path, 'w') as f:
    for i in range(len(images_path_HL)):
        f.write(images_path_HL[i] + ' ' + edges_path_HL[i] + '\n')

with open(HH_list_path, 'w') as f:
    for i in range(len(images_path_HH)):
        f.write(images_path_HH[i] + ' ' + edges_path_HH[i] + '\n')

#### Test database images